In [10]:
# connect to google drive first
!unzip /content/drive/MyDrive/RWF-2000/val.zip

Archive:  /content/drive/MyDrive/RWF-2000/val.zip
   creating: val/
   creating: val/Fight/
  inflating: val/Fight/0053dfcc7b545ae8c9dfdff35d61e684.npy  
  inflating: val/Fight/01d9e608534d2cfea1f90f3570b01b0f.npy  
  inflating: val/Fight/02b3b94531931845f98491055254604f.npy  
  inflating: val/Fight/02c105bc8f8b9ab6e8bbb856653f5d08.npy  
  inflating: val/Fight/05b16cf10104e6cb2c4ea3dc93f31e60.npy  
  inflating: val/Fight/07287214dc4600d1d09e34436fb1e255.npy  
  inflating: val/Fight/0799854f8c081a4765c3d7aa6a732cb6.npy  
  inflating: val/Fight/0ac04274a7afced05b2318151e889777.npy  
  inflating: val/Fight/0ac1c7849fa79a97a6a5d2a23f4dfb9e.npy  
  inflating: val/Fight/0eca98b455e76d4283ae5584490bb64a.npy  
  inflating: val/Fight/101737bb8bfb26098e5def7db88b0f84.npy  
  inflating: val/Fight/12952fbbb2fc32d78dd94bcf91f58d64.npy  
  inflating: val/Fight/150b87211ac84fdff38da4460d19337b.npy  
  inflating: val/Fight/17091729da6b7de0d1a0c2159935907e.npy  
  inflating: val/Fight/1a6a907d33341d481

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras import utils as np_utils
import os
import numpy as np

class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, directory, batch_size=1, shuffle=True, data_augmentation=True):
        # Initialize the params
        self.batch_size = batch_size
        self.directory = directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
        
    def search_data(self):
        X_path = []
        Y_dict = {}
        # list all kinds of sub-folders
        self.dirs = sorted(os.listdir(self.directory))
        one_hots = np_utils.to_categorical(range(len(self.dirs)))
        for i,folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory,folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path,file)
                # append the each file path, and keep its label  
                X_path.append(file_path)
                Y_dict[file_path] = one_hots[i]
        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = len(self.dirs)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(self.dirs):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
      
    def normalize(self, data):
        mean = np.mean(data)
        std = np.std(data)
        return (data-mean) / std
    
    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(m=video, axis=2)
        return video    
    
    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval 
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and 
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])     
        # calculate numer of padded frames and fix it 
        num_pad = target_frames - len(sampled_video)
        if num_pad>0:
            padding = [video[i] for i in range(-num_pad,0)]
            sampled_video += padding     
        # get sampled video
        return np.array(sampled_video, dtype=np.float32)
    
    def color_jitter(self,video):
        # range of s-component: 0-1
        # range of v component: 0-255
        s_jitter = np.random.uniform(-0.2,0.2)
        v_jitter = np.random.uniform(-30,30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
            s = hsv[...,1] + s_jitter
            v = hsv[...,2] + v_jitter
            s[s<0] = 0
            s[s>1] = 1
            v[v<0] = 0
            v[v>255] = 255
            hsv[...,1] = s
            hsv[...,2] = v
            video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        return video
        
    def load_data(self, path):
        data = np.load(path, mmap_mode='r')[...,:3]
        data = np.float32(data)
        # sampling 64 frames uniformly from the entire video
        data = self.uniform_sampling(video=data, target_frames=32)
#         data = np.resize(data, (64, 112, 112, 3))
        # whether to utilize the data augmentation
        if  self.data_aug:
            data = self.color_jitter(data)
            data = self.random_flip(data, prob=0.5)
        # normalize
        data = self.normalize(data)
        return data

In [ ]:
from tensorflow import keras
# replace with your path
model_path = '/content/drive/MyDrive/epoch-19-val_acc-0.87.h5'
model = keras.models.load_model(model_path)
val_path = '/content/val'
val_generator = DataGenerator(val_path,
                              batch_size=1, 
                              data_augmentation=False)
model.evaluate(val_generator)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
